# Start Gui

This notebook can be used to start the GUI and test out stuff.

In [1]:
# -*- coding: utf-8 -*-
"""
@author: Fabian Balzer

***
LICENSE:
    Copyright 2021 Fabian Balzer

   Licensed under the Apache License, Version 2.0 (the "License");
   you may not use this file except in compliance with the License.
   You may obtain a copy of the License at

     http://www.apache.org/licenses/LICENSE-2.0

   Unless required by applicable law or agreed to in writing, software
   distributed under the License is distributed on an "AS IS" BASIS,
   WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
   See the License for the specific language governing permissions and
   limitations under the License.
***

Code to open a GUI Dominion randomizer
"""
# %%

import sys
import random_kingdominion as rm
import PyQt5.QtWidgets as QW


def start_program():
    """A function to include everything needed to start the application"""
    # Check whether there is already a running QApplication (e.g. if running
    # from an IDE). This setup prevents crashes for the next run:
    qapp = QW.QApplication.instance()
    if not qapp:
        qapp = QW.QApplication(sys.argv)
    app = rm.UIMainWindow()  # creating the instance
    app.show()
    app.activateWindow()
    qapp.exec_()  # Start the Qt event loop



In [10]:

start_program()

[] Peril
[['Shy', 'Charm']]
['Shy'] Museum
[['Shy', 'Charm'], ['Cursed', 'Enlarge']]
['Shy', 'Cursed'] Battlefield
[['Shy', 'Charm'], ['Cursed', 'Enlarge']]
['Shy', 'Cursed'] Triumphal Arch
[['Shy', 'Charm'], ['Cursed', 'Enlarge'], ['Cheap', 'City Quarter']]
['Shy', 'Cursed', 'Cheap'] Shy
[['Cursed', 'Enlarge'], ['Cheap', 'City Quarter']]
['Cursed', 'Cheap'] Cursed
[['Cheap', 'City Quarter'], ['Pious', 'Villa']]
['Cheap', 'Pious'] Tomb
[['Cheap', 'City Quarter'], ['Pious', 'Villa']]
['Cheap', 'Pious'] Cheap
[['Pious', 'Villa'], ['Nearby', 'Silver Mine']]
['Pious', 'Nearby'] Pious
[['Nearby', 'Silver Mine']]
['Nearby'] Avoid
[['Nearby', 'Silver Mine']]
['Nearby'] Nearby
[]
[] Salt the Earth
[['Rich', 'Villa']]
['Rich'] Banquet
[['Rich', 'Villa'], ['Tireless', 'Charm']]
['Rich', 'Tireless'] Arena
[['Rich', 'Villa'], ['Tireless', 'Charm']]
['Rich', 'Tireless'] Delve
[['Rich', 'Villa'], ['Tireless', 'Charm']]
['Rich', 'Tireless'] Scrounge
[['Rich', 'Villa'], ['Tireless', 'Charm'], ['Friend

In [26]:
import numpy as np
config = rm.Custo()
# randomizer = rm.KingdomRandomizer(config)
# randomizer.randomize_new_kingdom()
rm.ALL_CARDS.memory_usage(deep=True).sum()
rm.ALL_CARDS.astype({"Expansion": "category"}).head().dtypes

Name                       object
Expansion                category
Types                      object
Cost                       object
Text                       object
Actions / Villagers        object
Cards                      object
Buys                       object
Coins / Coffers            object
Trash / Return             object
Exile                      object
Junk                       object
Gain                       object
Victory Points             object
ImagePath                  object
IsLandscape                  bool
IsOtherThing                 bool
IsInSupply                   bool
altvp_quality               int64
attack_quality              int64
draw_quality                int64
gain_quality                int64
interactivity_quality       int64
thinning_quality            int64
village_quality             int64
attack_types               object
gain_types                 object
thinning_types             object
CardAmount                 object
IsAlly        

In [6]:
import random_kingdominion as rk
import os
import pandas as pd
import numpy as np
from typing import Sequence


def read_dataframe_from_file(fpath: str, eval_lists=False):
    """Read a dataframe"""
    if os.path.isfile(fpath):
        df = pd.read_csv(fpath, sep=";", header=0)
        if eval_lists:
            for colname in df.columns:
                if "type" in colname.lower():
                    # Make sure we properly handle lists
                    df[colname] = df[colname].apply(eval)
    else:
        raise FileNotFoundError(
            2, "Couldn't find the raw card data file, please download it first."
        )
    return df
from functools import reduce
df = read_dataframe_from_file(rk.FPATH_CARD_DATA, True)
names = rk.ALL_CSOS["Name"].sample(200).values

535 µs ± 3.39 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
5.47 ms ± 99.3 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
